# Frustration Agent Testing Notebook

This notebook provides a user-friendly interface for testing the Frustration Agent with chatbot interactions.
It reads output from chatbot_tester.ipynb and analyzes customer frustration levels in conversations.

## Features:
- Load and edit Frustration Agent configuration settings
- Import chatbot conversation results from chatbot_tester.ipynb
- Process each conversation turn through the Frustration Agent
- Generate frustration scores, sentiment analysis, and escalation recommendations
- Track frustration patterns and escalation trends across conversations
- Analyze employee wellbeing impact and routing recommendations
- Export enhanced results with frustration and sentiment metrics

## Getting Started:
1. Run cells in order from top to bottom
2. Edit configuration values as needed
3. Load chatbot conversation results from chatbot_tester exports
4. Review conversations before frustration analysis
5. Run frustration detection and review detailed results

## Input Requirements:
Load conversation results exported from chatbot_tester.ipynb containing:
- Customer queries with potential frustration indicators
- Conversation metadata and turn-by-turn interactions
- Customer types, complexity levels, and interaction history

In [1]:
# Import required libraries
import json
import os
import sys
from datetime import datetime, timedelta
from pathlib import Path
import uuid

import ipywidgets as widgets
import pandas as pd
from ruamel.yaml import YAML
from IPython.display import display, HTML, clear_output

# Set the working directory to the root of the project
os.chdir('/workspace')

# Add workspace to path for imports
sys.path.insert(0, '/workspace')

# Import our system components
from src.nodes.frustration_agent import FrustrationAgentNode
from src.core.config import ConfigManager
from src.core.context_manager import SQLiteContextProvider
from src.interfaces.core.state_schema import HybridSystemState
from src.interfaces.core.context import ContextEntry

print("✅ All libraries imported successfully!")
print("Ready to start testing the Frustration Agent.")

✅ All libraries imported successfully!
Ready to start testing the Frustration Agent.


## Step 1: Load Configuration Settings

The following cell loads the current configuration for the Frustration Agent.
You can edit these values to customize the agent's frustration detection behavior.

In [2]:
# Load configuration from files and create temporary editable copies
config_base_path = Path('/workspace/config')
agent_config_path = config_base_path / 'agents' / 'frustration_agent'
temp_config_dir = Path('/tmp/frustration_agent_configs')

def load_and_create_temp_configs():
    """Load all configuration files and create temporary editable copies with comments preserved"""
    configs = {}
    
    # Create YAML instance for comment preservation
    yaml = YAML()
    yaml.preserve_quotes = True
    yaml.default_flow_style = False
    
    # Create temp directory
    temp_config_dir.mkdir(exist_ok=True)
    
    # Load original files for parsing (to access data)
    with open(agent_config_path / 'config.yaml', 'r') as f:
        configs['agent'] = yaml.load(f)
    
    with open(agent_config_path / 'prompts.yaml', 'r') as f:
        configs['prompts'] = yaml.load(f)
    
    with open(agent_config_path / 'models.yaml', 'r') as f:
        configs['models'] = yaml.load(f)
    
    # Load shared models for reference
    with open(config_base_path / 'shared' / 'models.yaml', 'r') as f:
        configs['shared_models'] = yaml.load(f)
    
    # Create temp file paths
    temp_agent_path = temp_config_dir / 'config.yaml'
    temp_prompts_path = temp_config_dir / 'prompts.yaml'
    temp_models_path = temp_config_dir / 'models.yaml'
    
    # Copy original files to temp directory to preserve comments and formatting
    import shutil
    shutil.copy2(agent_config_path / 'config.yaml', temp_agent_path)
    shutil.copy2(agent_config_path / 'prompts.yaml', temp_prompts_path)  
    shutil.copy2(agent_config_path / 'models.yaml', temp_models_path)
    
    return configs, {
        'agent_config': temp_agent_path,
        'prompts_config': temp_prompts_path,
        'models_config': temp_models_path
    }

# Load configurations and create temp files
configs, temp_file_paths = load_and_create_temp_configs()

print("📁 Configuration files loaded and temporary copies created with comments preserved!")
print(f"Agent name: {configs['agent']['agent']['name']}")
print(f"Agent version: {configs['agent']['agent']['version']}")

# Get preferred model from models config
preferred_model = "Unknown"
if 'primary_model' in configs['models']:
    preferred_model = configs['models']['primary_model']
elif 'preferred' in configs['models']:
    preferred_model = configs['models']['preferred']

print(f"Preferred model: {preferred_model}")

# Display key frustration thresholds
frustration_thresholds = configs['agent']['settings']['frustration_thresholds']
print(f"\n😤 Frustration Detection Thresholds:")
print(f"  Critical: {frustration_thresholds['critical']}")
print(f"  High: {frustration_thresholds['high']}")
print(f"  Moderate: {frustration_thresholds['moderate']}")

# Display intervention threshold
intervention_threshold = configs['agent']['settings']['intervention_threshold']
print(f"  Intervention trigger: {intervention_threshold}")

print(f"\n💾 Temporary config files created at:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")
print(f"\n💡 These temp files retain original comments and can be edited directly in Step 2.")

📁 Configuration files loaded and temporary copies created with comments preserved!
Agent name: frustration_agent
Agent version: 1.0.0
Preferred model: anthropic_general_standard

😤 Frustration Detection Thresholds:
  Critical: 8.0
  High: 6.0
  Moderate: 3.0
  Intervention trigger: high

💾 Temporary config files created at:
  agent_config: /tmp/frustration_agent_configs/config.yaml
  prompts_config: /tmp/frustration_agent_configs/prompts.yaml
  models_config: /tmp/frustration_agent_configs/models.yaml

💡 These temp files retain original comments and can be edited directly in Step 2.


## Step 2: Editable Configuration Settings

Edit these settings to customize how the Frustration Agent detects and analyzes customer frustration.
These variables map directly to the configuration files and can be exported later.

In [3]:
# Display and edit configuration files in separate windows

def load_config_file_contents():
    """Load current config file contents from temp files"""
    with open(temp_file_paths['agent_config'], 'r') as f:
        agent_config_content = f.read()
    with open(temp_file_paths['prompts_config'], 'r') as f:
        prompts_config_content = f.read()
    with open(temp_file_paths['models_config'], 'r') as f:
        models_config_content = f.read()
    
    return agent_config_content, prompts_config_content, models_config_content

# Load current config file contents
agent_config_content, prompts_config_content, models_config_content = load_config_file_contents()

print("⚙️ Frustration Agent Configuration File Editor")
print("Edit the YAML configuration files below and use the Save buttons to apply changes.")
print("Changes are saved to temporary files and will be used in frustration detection.\n")

# Create text areas for each config file
print("📄 1. Frustration Agent Configuration (config.yaml)")
print("Contains: frustration thresholds, intervention settings, pattern detection rules")

agent_config_editor = widgets.Textarea(
    value=agent_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='350px'),
    style={'description_width': '0px'}
)

def save_agent_config(button):
    """Save agent config changes with comments preserved"""
    try:
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        yaml.load(agent_config_editor.value)
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['agent_config'], 'w') as f:
            f.write(agent_config_editor.value)
        
        print("✅ Frustration Agent config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in agent config: {e}")

agent_save_btn = widgets.Button(description="Save Agent Config", button_style='success')
agent_save_btn.on_click(save_agent_config)

display(agent_config_editor)
display(agent_save_btn)

print("\n📄 2. Frustration Detection Prompts Configuration (prompts.yaml)")
print("Contains: system prompts, frustration analysis criteria, pattern detection templates")

prompts_config_editor = widgets.Textarea(
    value=prompts_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='300px'),
    style={'description_width': '0px'}
)

def save_prompts_config(button):
    """Save prompts config changes with comments preserved"""
    try:
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        yaml.load(prompts_config_editor.value)
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['prompts_config'], 'w') as f:
            f.write(prompts_config_editor.value)
        
        print("✅ Prompts config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in prompts config: {e}")

prompts_save_btn = widgets.Button(description="Save Prompts Config", button_style='success')
prompts_save_btn.on_click(save_prompts_config)

display(prompts_config_editor)
display(prompts_save_btn)

print("\n📄 3. Frustration Analysis Models Configuration (models.yaml)")
print("Contains: preferred models for frustration detection, pattern analysis")

# Show available model aliases from shared models config
def display_available_models():
    """Display available model aliases and their actual models"""
    try:
        shared_models = configs['shared_models']
        
        # Extract model aliases and models sections
        model_aliases = shared_models.get('model_aliases', {})
        models = shared_models.get('models', {})
        
        if not model_aliases:
            print("❌ No model aliases found in shared configuration")
            return
            
        print("\n🔍 Available Model Aliases for Frustration Detection:")
        print("Use these aliases in your models configuration below:\n")
        
        # Group by provider for better organization
        providers = {}
        for alias, actual_model_name in model_aliases.items():
            # Get model details from models section
            model_details = models.get(actual_model_name, {})
            provider = model_details.get('type', 'unknown')
            description = model_details.get('description', '')
            
            if provider not in providers:
                providers[provider] = []
            providers[provider].append({
                'alias': alias,
                'model_name': actual_model_name,
                'description': description
            })
        
        # Display by provider with frustration detection recommendations
        for provider, provider_models in providers.items():
            print(f"📡 {provider.upper()} Provider:")
            for model in provider_models:
                desc = f" - {model['description']}" if model['description'] else ""
                print(f"  • {model['alias']} → {model['model_name']}{desc}")
            print()
        
        # Show current configuration
        try:
            yaml = YAML()
            yaml.preserve_quotes = True
            current_models_config = yaml.load(models_config_content)
            current_preferred = current_models_config.get('primary_model', current_models_config.get('preferred', 'unknown'))
            
            # Show model preferences for different tasks
            model_preferences = current_models_config.get('model_preferences', {})
        except:
            current_preferred = 'unknown'
            model_preferences = {}
        
        print(f"📋 Current Models Configuration:")
        print(f"  Primary model: {current_preferred}")
        if model_preferences:
            print(f"  Task-specific preferences:")
            for task, prefs in model_preferences.items():
                print(f"    {task}: {prefs.get('primary', 'unknown')}")
        print()
        
        print("💡 Frustration Detection Model Recommendations:")
        print("  • Very low temperature models (0.1-0.2) for consistent sentiment analysis")
        print("  • Standard models sufficient for basic frustration detection")
        print("  • Budget models acceptable for rule-based pattern matching")
        print("  • Reasoning models beneficial for complex emotional analysis")
        print()
        
    except Exception as e:
        print(f"❌ Error loading available models: {e}")
        print("Continuing with models configuration editor...\n")

# Display available models before showing the editor
display_available_models()

models_config_editor = widgets.Textarea(
    value=models_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': '0px'}
)

def save_models_config(button):
    """Save models config changes with comments preserved"""
    try:
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        parsed_config = yaml.load(models_config_editor.value)
        
        # Additional validation for model aliases
        if isinstance(parsed_config, dict):
            preferred = parsed_config.get('primary_model') or parsed_config.get('preferred')
            
            # Get available aliases
            model_aliases = configs['shared_models'].get('model_aliases', {})
            
            # Check if preferred model exists
            if preferred and preferred not in model_aliases:
                print(f"⚠️ Warning: Preferred model '{preferred}' not found in available model aliases")
            
            # Check model preferences
            model_preferences = parsed_config.get('model_preferences', {})
            for task, prefs in model_preferences.items():
                task_primary = prefs.get('primary')
                if task_primary and task_primary not in model_aliases:
                    print(f"⚠️ Warning: {task} primary model '{task_primary}' not found in available model aliases")
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['models_config'], 'w') as f:
            f.write(models_config_editor.value)
        
        print("✅ Models config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in models config: {e}")

models_save_btn = widgets.Button(description="Save Models Config", button_style='success')
models_save_btn.on_click(save_models_config)

display(models_config_editor)
display(models_save_btn)

# Save All button for convenience
def save_all_configs(button):
    """Save all config changes at once"""
    save_agent_config(None)
    save_prompts_config(None)
    save_models_config(None)

print("\n💾 Save All Changes")
save_all_btn = widgets.Button(description="Save All Configs", button_style='info')
save_all_btn.on_click(save_all_configs)
display(save_all_btn)

print(f"\n💾 Temp config files location:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")

print("\n💡 Frustration Detection Tips:")
print("  • Lower critical/high thresholds = more sensitive frustration detection")
print("  • intervention_threshold controls when to escalate to humans")
print("  • Lower temperature = more consistent frustration scoring")
print("  • Edit frustration_indicators to customize keyword detection")
print("  • Employee protection settings help manage agent workload")

⚙️ Frustration Agent Configuration File Editor
Edit the YAML configuration files below and use the Save buttons to apply changes.
Changes are saved to temporary files and will be used in frustration detection.

📄 1. Frustration Agent Configuration (config.yaml)
Contains: frustration thresholds, intervention settings, pattern detection rules


Textarea(value='# Frustration Agent Configuration\n# Responsibility: Analyze customer comments to detect frust…

Button(button_style='success', description='Save Agent Config', style=ButtonStyle())


📄 2. Frustration Detection Prompts Configuration (prompts.yaml)
Contains: system prompts, frustration analysis criteria, pattern detection templates


Textarea(value='# Frustration Agent Prompts\n\nsystem: |\n  You are a Frustration Detection Agent responsible …

...

Textarea(value='primary_model: "anthropic_general_standard"\n\nmodel_preferences:\n  frustration_analysis:\n  …

Button(button_style='success', description='Save Models Config', style=ButtonStyle())


💾 Save All Changes


Button(button_style='info', description='Save All Configs', style=ButtonStyle())


💾 Temp config files location:
  agent_config: /tmp/frustration_agent_configs/config.yaml
  prompts_config: /tmp/frustration_agent_configs/prompts.yaml
  models_config: /tmp/frustration_agent_configs/models.yaml

💡 Frustration Detection Tips:
  • Lower critical/high thresholds = more sensitive frustration detection
  • intervention_threshold controls when to escalate to humans
  • Lower temperature = more consistent frustration scoring
  • Edit frustration_indicators to customize keyword detection
  • Employee protection settings help manage agent workload


## Step 3: Load Chatbot Conversation Results

Load conversation results from chatbot_tester.ipynb exports for frustration analysis.

In [4]:
# File upload widget for loading conversation results
file_upload = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='Upload conversation results:'
)

# Instructions for file format
print("📝 Load Chatbot Conversation Results for Frustration Analysis")
print("\n💡 How to get conversation data:")
print("1. Run chatbot_tester.ipynb and export conversation results")
print("2. Upload the exported JSON file below")
print("3. The file should contain conversation turns with customer queries")
print("\n📄 Expected JSON format:")
print("- Array of conversations with conversation_history containing:")
print("- Each turn: {'customer_query': '...', 'chatbot_response': '...', 'turn_number': N}")
print("- Customer queries will be analyzed for frustration indicators")
print("- Conversation patterns will be tracked for escalation trends")
print("\n📁 Upload your conversation results file:")
display(file_upload)

📝 Load Chatbot Conversation Results for Frustration Analysis

💡 How to get conversation data:
1. Run chatbot_tester.ipynb and export conversation results
2. Upload the exported JSON file below
3. The file should contain conversation turns with customer queries

📄 Expected JSON format:
- Array of conversations with conversation_history containing:
- Each turn: {'customer_query': '...', 'chatbot_response': '...', 'turn_number': N}
- Customer queries will be analyzed for frustration indicators
- Conversation patterns will be tracked for escalation trends

📁 Upload your conversation results file:


FileUpload(value=(), accept='.json', description='Upload conversation results:')

In [5]:
# Load conversation results from uploaded file and prepare context data
conversation_data = []

def load_conversations_from_file(file_content, filename):
    """Load conversations from uploaded JSON file with enhanced format detection"""
    try:
        # Handle different content types
        if isinstance(file_content, memoryview):
            content_bytes = file_content.tobytes()
        elif hasattr(file_content, 'decode'):
            content_bytes = file_content
        else:
            content_bytes = str(file_content).encode('utf-8')
        
        # Decode to string and parse JSON
        data = json.loads(content_bytes.decode('utf-8'))
        
        # Handle different JSON formats with enhanced detection
        conversations = []
        
        if isinstance(data, dict):
            # Check for different export formats from chatbot_tester
            if 'conversation_results' in data:
                # New chatbot_tester export format
                conversations = data['conversation_results']
            elif 'conversations' in data:
                # Standard conversations format
                conversations = data['conversations']
            elif 'results' in data:
                # Results array format
                conversations = data['results']
            elif 'metadata' in data and 'conversations' in data:
                # Metadata + conversations format
                conversations = data['conversations']
            elif any(key in data for key in ['conversation_history', 'customer_type', 'original_question']):
                # Single conversation object
                conversations = [data]
            else:
                # Try to extract any array-like data
                array_fields = [v for v in data.values() if isinstance(v, list)]
                if array_fields:
                    conversations = array_fields[0]
                else:
                    print(f"⚠️ Unknown dictionary format. Keys found: {list(data.keys())}")
                    return []
        elif isinstance(data, list):
            # Direct list of conversations
            conversations = data
        else:
            print(f"❌ Unexpected data format: {type(data)}")
            return []
        
        # Validate that we have actual conversation data
        if not conversations:
            print("❌ No conversations found in the uploaded file")
            return []
        
        # Validate conversation structure
        valid_conversations = []
        for i, conv in enumerate(conversations):
            if not isinstance(conv, dict):
                print(f"⚠️ Skipping invalid conversation {i+1}: not a dictionary")
                continue
            
            # Check for required fields or conversation history
            has_history = 'conversation_history' in conv and conv['conversation_history']
            has_single_turn = any(field in conv for field in ['query', 'customer_query', 'original_question'])
            
            if not (has_history or has_single_turn):
                print(f"⚠️ Skipping conversation {i+1}: no conversation history or query found")
                continue
                
            valid_conversations.append(conv)
        
        if not valid_conversations:
            print("❌ No valid conversations found after validation")
            return []
        
        print(f"✅ Successfully loaded {len(valid_conversations)} valid conversations from {len(conversations)} total entries")
        return valid_conversations
            
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print("Please ensure the file is valid JSON format")
        return []
    except Exception as e:
        print(f"❌ Error loading file '{filename}': {e}")
        print("Please check the file format and try again")
        return []

def extract_conversation_turns_with_context(conversations):
    """Extract individual turns from conversations and build context for frustration analysis"""
    turns = []
    
    for conv_idx, conversation in enumerate(conversations):
        conv_id = conversation.get('id', conversation.get('conversation_id', conv_idx + 1))
        customer_type = conversation.get('customer_type', 'unknown')
        complexity = conversation.get('complexity', 'medium')
        
        # Get conversation history with multiple format support
        history = conversation.get('conversation_history', [])
        
        # If no conversation_history, try to extract from other fields
        if not history:
            # Check for single turn conversation
            query = (conversation.get('query') or 
                    conversation.get('customer_query') or 
                    conversation.get('original_question'))
            response = (conversation.get('ai_response') or 
                       conversation.get('chatbot_response') or 
                       conversation.get('response'))
            
            if query and response:
                history = [{
                    'turn_number': 1,
                    'customer_query': query,
                    'chatbot_response': response
                }]
        
        # Skip conversations with no extractable history
        if not history:
            print(f"⚠️ Skipping conversation {conv_id}: no turns found")
            continue
        
        # Extract turns from conversation history and build context
        for turn_idx, turn in enumerate(history):
            # Skip invalid turns
            if not isinstance(turn, dict):
                continue
                
            customer_query = turn.get('customer_query', turn.get('query', ''))
            chatbot_response = turn.get('chatbot_response', turn.get('response', ''))
            
            if not customer_query:
                continue
            
            # Create simulated timestamps for frustration pattern analysis
            base_timestamp = datetime.now() - timedelta(hours=1) + timedelta(minutes=turn_idx * 5)
            
            turn_data = {
                'conversation_id': conv_id,
                'user_id': f'user_{conv_id}',
                'session_id': f'session_{conv_id}',
                'turn_number': turn.get('turn_number', turn_idx + 1),
                'timestamp': base_timestamp,
                'customer_type': customer_type,
                'complexity': complexity,
                'customer_query': customer_query,
                'chatbot_response': chatbot_response,
                'original_customer_satisfaction': turn.get('customer_satisfaction'),
                'conversation_metadata': {
                    'total_turns': len(history),
                    'final_outcome': conversation.get('final_outcome'),
                    'original_question': conversation.get('original_question', ''),
                    'turn_index': turn_idx,  # Position in conversation
                    'is_first_turn': turn_idx == 0,
                    'is_last_turn': turn_idx == len(history) - 1,
                },
                # Previous turns in conversation for pattern analysis
                'previous_turns': history[:turn_idx] if turn_idx > 0 else []
            }
            turns.append(turn_data)
    
    return turns

def simulate_context_history(turn_data, context_provider):
    """Simulate conversation history in context provider for pattern analysis"""
    try:
        # Add previous turns to context for pattern analysis
        for prev_turn_idx, prev_turn in enumerate(turn_data['previous_turns']):
            # Create context entries for previous queries
            prev_timestamp = turn_data['timestamp'] - timedelta(minutes=(len(turn_data['previous_turns']) - prev_turn_idx) * 5)
            
            query_entry = ContextEntry(
                entry_id=f"{turn_data['conversation_id']}_{prev_turn_idx + 1}_query",
                user_id=turn_data['user_id'],
                session_id=turn_data['session_id'],
                timestamp=prev_timestamp,
                entry_type="query",
                content=prev_turn.get('customer_query', ''),
                metadata={
                    'turn_number': prev_turn_idx + 1,
                    'conversation_id': turn_data['conversation_id']
                }
            )
            context_provider.save_context_entry(query_entry)
            
            # Create context entries for previous responses
            response_entry = ContextEntry(
                entry_id=f"{turn_data['conversation_id']}_{prev_turn_idx + 1}_response",
                user_id=turn_data['user_id'],
                session_id=turn_data['session_id'],
                timestamp=prev_timestamp + timedelta(seconds=30),
                entry_type="response",
                content=prev_turn.get('chatbot_response', ''),
                metadata={
                    'turn_number': prev_turn_idx + 1,
                    'conversation_id': turn_data['conversation_id']
                }
            )
            context_provider.save_context_entry(response_entry)
            
    except Exception as e:
        print(f"Warning: Could not simulate context history: {e}")

# Process uploaded file with enhanced error handling
if file_upload.value:
    uploaded_file = None
    filename = None
    file_content = None
    
    # Handle different file upload widget formats
    try:
        if isinstance(file_upload.value, tuple) and len(file_upload.value) > 0:
            uploaded_file = file_upload.value[0]
            filename = uploaded_file['name']
            file_content = uploaded_file['content']
        elif isinstance(file_upload.value, dict) and len(file_upload.value) > 0:
            uploaded_file = list(file_upload.value.values())[0]
            filename = uploaded_file['metadata']['name']
            file_content = uploaded_file['content']
        else:
            print(f"❌ Unable to read uploaded file format: {type(file_upload.value)}")
    except Exception as e:
        print(f"❌ Error accessing uploaded file: {e}")
    
    if uploaded_file and filename and file_content is not None:
        print(f"📁 Loading conversations from: {filename}")
        
        raw_conversations = load_conversations_from_file(file_content, filename)
        
        if raw_conversations:
            conversation_data = extract_conversation_turns_with_context(raw_conversations)
            
            if conversation_data:
                print(f"✅ Loaded {len(conversation_data)} conversation turns from {len(raw_conversations)} conversations")
                
                # Display conversation statistics
                df_preview = pd.DataFrame(conversation_data)
                print(f"\n📊 Conversation Turn Distribution:")
                print(f"  Total turns: {len(conversation_data)}")
                print(f"  Unique conversations: {df_preview['conversation_id'].nunique()}")
                print(f"  Customer types: {dict(df_preview['customer_type'].value_counts())}")
                print(f"  Complexity levels: {dict(df_preview['complexity'].value_counts())}")
                
                # Analyze frustration indicators in queries
                print(f"\n🔍 Preliminary Frustration Indicator Analysis:")
                high_frustration_keywords = ['angry', 'frustrated', 'terrible', 'awful', 'ridiculous', 'unacceptable', 'furious', 'livid', 'outraged']
                moderate_frustration_keywords = ['annoyed', 'disappointed', 'upset', 'confused', 'stuck', 'concerned', 'worried', 'bothered']
                escalation_keywords = ['manager', 'supervisor', 'complaint', 'cancel', 'refund', 'speak to someone', 'escalate', 'higher up']
                
                high_frustration_turns = 0
                moderate_frustration_turns = 0
                escalation_turns = 0
                
                for turn in conversation_data:
                    query_lower = turn['customer_query'].lower()
                    if any(keyword in query_lower for keyword in high_frustration_keywords):
                        high_frustration_turns += 1
                    if any(keyword in query_lower for keyword in moderate_frustration_keywords):
                        moderate_frustration_turns += 1
                    if any(keyword in query_lower for keyword in escalation_keywords):
                        escalation_turns += 1
                
                print(f"  Potential high frustration turns: {high_frustration_turns} ({high_frustration_turns/len(conversation_data)*100:.1f}%)")
                print(f"  Potential moderate frustration turns: {moderate_frustration_turns} ({moderate_frustration_turns/len(conversation_data)*100:.1f}%)")
                print(f"  Potential escalation requests: {escalation_turns} ({escalation_turns/len(conversation_data)*100:.1f}%)")
                
                # Show sample turns
                print(f"\n📋 Sample Conversation Turns for Frustration Analysis:")
                for i, turn in enumerate(conversation_data[:3]):
                    query_preview = turn['customer_query'][:80] + "..." if len(turn['customer_query']) > 80 else turn['customer_query']
                    print(f"  Turn {turn['conversation_id']}.{turn['turn_number']} [{turn['customer_type']}]:")
                    print(f"    Query: {query_preview}")
                    print(f"    Context: {len(turn['previous_turns'])} previous turns in conversation")
                if len(conversation_data) > 3:
                    print(f"  ... and {len(conversation_data) - 3} more turns")
            else:
                print("❌ No valid conversation turns extracted from the loaded data")
        else:
            print("❌ Failed to load conversations from file")
    else:
        print("❌ Error accessing uploaded file data")
else:
    print("⚠️ Please upload a JSON file with conversation results from chatbot_tester.ipynb.")
    print("\n💡 Supported file formats:")
    print("  • chatbot_tester.ipynb export files (conversation_results format)")
    print("  • Standard conversation JSON files")
    print("  • Any JSON file with conversation history or query/response pairs")

📁 Loading conversations from: chatbot_agent_output.json
✅ Successfully loaded 20 valid conversations from 20 total entries
✅ Loaded 66 conversation turns from 20 conversations

📊 Conversation Turn Distribution:
  Total turns: 66
  Unique conversations: 20
  Customer types: {'confused': np.int64(25), 'urgent': np.int64(18), 'normal': np.int64(12), 'frustrated': np.int64(11)}
  Complexity levels: {'simple': np.int64(24), 'medium': np.int64(21), 'complex': np.int64(21)}

🔍 Preliminary Frustration Indicator Analysis:
  Potential high frustration turns: 5 (7.6%)
  Potential moderate frustration turns: 1 (1.5%)
  Potential escalation requests: 3 (4.5%)

📋 Sample Conversation Turns for Frustration Analysis:
  Turn 1.1 [frustrated]:
    Query: Why did my premium increase by $200? This is ridiculous - I haven't had any clai...
    Context: 0 previous turns in conversation
  Turn 1.2 [frustrated]:
    Query: "I can't believe you are giving me this runaround! I demand to know why my premi...
    

## Step 4: Initialize Frustration Agent

Create the Frustration Agent instance with the configured settings.

In [6]:
# Initialize Frustration Agent with temporary configuration
def initialize_frustration_agent():
    """Initialize Frustration Agent using temporary configuration files"""
    try:
        # Create a complete temporary config structure that ConfigManager expects
        temp_base_config_dir = temp_config_dir.parent / 'temp_frustration_config'
        temp_base_config_dir.mkdir(exist_ok=True)
        
        # Create the expected directory structure
        temp_agents_dir = temp_base_config_dir / 'agents' / 'frustration_agent'
        temp_shared_dir = temp_base_config_dir / 'shared'
        temp_agents_dir.mkdir(parents=True, exist_ok=True)
        temp_shared_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy agent-specific configs to the expected location
        import shutil
        shutil.copy2(temp_file_paths['agent_config'], temp_agents_dir / 'config.yaml')
        shutil.copy2(temp_file_paths['prompts_config'], temp_agents_dir / 'prompts.yaml')
        shutil.copy2(temp_file_paths['models_config'], temp_agents_dir / 'models.yaml')
        
        # Copy shared configs that agents might need
        original_shared_dir = config_base_path / 'shared'
        for shared_file in ['models.yaml', 'system.yaml', 'providers.yaml']:
            if (original_shared_dir / shared_file).exists():
                shutil.copy2(original_shared_dir / shared_file, temp_shared_dir / shared_file)
        
        # Copy main config.yaml if needed
        if (config_base_path / 'config.yaml').exists():
            shutil.copy2(config_base_path / 'config.yaml', temp_base_config_dir / 'config.yaml')
        
        # Create ConfigManager with the complete structure
        config_manager = ConfigManager(config_dir=str(temp_base_config_dir))
        
        # Initialize context provider (using in-memory for testing)
        context_provider = SQLiteContextProvider(db_path=":memory:")
        
        # Create Frustration Agent
        frustration_agent = FrustrationAgentNode(
            config_manager=config_manager,
            context_provider=context_provider
        )
        
        print("✅ Frustration Agent initialized successfully!")
        print(f"  Agent config: {frustration_agent.agent_config.name if frustration_agent.agent_config else 'Unknown'}")
        print(f"  LLM provider: {frustration_agent.llm_provider.model_name if frustration_agent.llm_provider else 'None'}")
        
        return frustration_agent, context_provider
        
    except Exception as e:
        print(f"❌ Error initializing Frustration Agent: {e}")
        import traceback
        print("Full traceback:")
        traceback.print_exc()
        return None, None

frustration_agent = None
context_provider = None

print("🔧 Initializing Frustration Agent with configured settings...")

# Initialize the frustration agent
frustration_agent, context_provider = initialize_frustration_agent()

if frustration_agent:
    # Display agent configuration
    if frustration_agent.agent_config:
        settings = frustration_agent.agent_config.settings
        print(f"\n📋 Frustration Agent Configuration:")
        print(f"  Frustration thresholds:")
        frustration_thresholds = settings.get('frustration_thresholds', {})
        for threshold_name, threshold_value in frustration_thresholds.items():
            print(f"    {threshold_name}: {threshold_value}")
        
        print(f"  Intervention threshold: {settings.get('intervention_threshold', 'high')}")
        
        analysis_settings = settings.get('analysis', {})
        print(f"  Analysis settings:")
        print(f"    Use LLM analysis: {analysis_settings.get('use_llm_analysis', True)}")
        print(f"    Track frustration progression: {analysis_settings.get('track_frustration_progression', True)}")
        print(f"    Consider interaction history: {analysis_settings.get('consider_interaction_history', True)}")
    
    # Show frustration indicators
    if hasattr(frustration_agent, 'frustration_indicators'):
        indicators = frustration_agent.frustration_indicators
        print(f"\n🔍 Frustration Detection Indicators:")
        print(f"  High frustration keywords: {len(indicators.get('high_frustration', []))}")
        print(f"  Moderate frustration keywords: {len(indicators.get('moderate_frustration', []))}")
        print(f"  Escalation phrases: {len(indicators.get('escalation_phrases', []))}")
        print(f"  Urgency indicators: {len(indicators.get('urgency_indicators', []))}")
    
    print(f"\n🚀 Frustration Agent is ready for conversation analysis!")
else:
    print(f"\n❌ Frustration Agent initialization failed. Please check configuration and try again.")

🔧 Initializing Frustration Agent with configured settings...
✅ 20:47:02.398 [INFO    ] context_manager | SQLite context provider initialized | operation=__init__
✅ 20:47:02.431 [INFO    ] factory         | Attempting to create provider: anthropic_general_standard
✅ Creating LLM provider: anthropic_general_standard → claude-3-5-sonnet-20241022 (anthropic)
✅ 20:47:02.434 [INFO    ] claude-3-5-sonnet-20241022 | Initializing LLM provider | model_name=claude-3-5-sonnet-20241022
✅ 20:47:02.438 [INFO    ] claude-3-5-sonnet-20241022 | LLM provider initialized successfully | model_name=claude-3-5-sonnet-20241022
✅ 20:47:02.439 [INFO    ] factory         | Successfully created provider: anthropic_general_standard
✅ 20:47:02.440 [INFO    ] frustration_agent | Frustration Agent LLM provider initialized | operation=initialize_llm_provider model_name=claude-3-5-sonnet-20241022
✅ Frustration Agent initialized successfully!
  Agent config: frustration_agent
  LLM provider: claude-3-5-sonnet-20241022



## Step 5: Run Frustration Analysis

Process each conversation turn through the Frustration Agent to generate frustration scores and intervention recommendations.

In [7]:
# Run frustration analysis on all conversation turns
frustration_results = []

def create_state_for_turn(turn_data):
    """Create HybridSystemState for a conversation turn"""
    return HybridSystemState({
        "query_id": f"frustration_test_{turn_data['conversation_id']}_{turn_data['turn_number']}",
        "user_id": turn_data['user_id'],
        "session_id": turn_data['session_id'],
        "timestamp": turn_data['timestamp'],
        "query": turn_data['customer_query'],
        "customer_type": turn_data['customer_type'],
        "complexity": turn_data['complexity'],
        "conversation_metadata": turn_data['conversation_metadata']
    })

def analyze_conversation_frustration(conversation_turns):
    """Run frustration analysis on conversation turns"""
    if not frustration_agent:
        print("❌ Frustration Agent not initialized. Please run the previous step.")
        return []
    
    if not conversation_turns:
        print("❌ No conversation turns to analyze. Please load conversation data first.")
        return []
    
    print(f"😤 Running frustration analysis on {len(conversation_turns)} conversation turns...")
    
    analyzed_turns = []
    
    for i, turn in enumerate(conversation_turns):
        try:
            # Simulate context history for pattern analysis
            simulate_context_history(turn, context_provider)
            
            # Create state for this turn
            state = create_state_for_turn(turn)
            
            print(f"\\rProcessing turn {i+1}/{len(conversation_turns)} (Conv {turn['conversation_id']}.{turn['turn_number']})...", end='', flush=True)
            
            # Run frustration analysis
            result_state = frustration_agent(state)
            
            # Extract frustration analysis results
            frustration_analysis = result_state.get('frustration_analysis', {})
            frustration_intervention_needed = result_state.get('frustration_intervention_needed', False)
            next_action = result_state.get('next_action', 'continue')
            
            # Create enhanced turn data with frustration metrics
            enhanced_turn = {
                **turn,  # Original turn data
                'frustration_analysis': frustration_analysis,
                'frustration_intervention_needed': frustration_intervention_needed,
                'next_action': next_action,
                'frustration_score': frustration_analysis.get('overall_score', 0.0),
                'frustration_level': frustration_analysis.get('overall_level', 'low'),
                'frustration_confidence': frustration_analysis.get('confidence', 0.0),
                'contributing_factors': frustration_analysis.get('contributing_factors', []),
                'current_query_score': frustration_analysis.get('current_analysis', {}).get('current_query_score', 0.0),
                'pattern_score': frustration_analysis.get('history_analysis', {}).get('pattern_score', 0.0),
                'escalation_trend': frustration_analysis.get('history_analysis', {}).get('escalation_trend', 'stable'),
                'analysis_timestamp': datetime.now().isoformat()
            }
            
            analyzed_turns.append(enhanced_turn)
            
        except Exception as e:
            print(f"\\n❌ Error analyzing turn {turn['conversation_id']}.{turn['turn_number']}: {e}")
            # Add turn with error information
            error_turn = {
                **turn,
                'frustration_analysis': {'error': str(e)},
                'frustration_score': 0.0,
                'frustration_level': 'error',
                'frustration_confidence': 0.0,
                'contributing_factors': [f'Analysis error: {str(e)}'],
                'frustration_intervention_needed': False,
                'analysis_timestamp': datetime.now().isoformat()
            }
            analyzed_turns.append(error_turn)
    
    print(f"\\n✅ Frustration analysis completed for {len(analyzed_turns)} turns!")
    return analyzed_turns

# Run analysis if we have data and agent
if conversation_data and frustration_agent:
    print("🚀 Starting frustration analysis process...")
    frustration_results = analyze_conversation_frustration(conversation_data)
    
    if frustration_results:
        # Display summary statistics
        df_results = pd.DataFrame(frustration_results)
        
        print(f"\\n📊 Frustration Analysis Summary:")
        print(f"  Total turns analyzed: {len(frustration_results)}")
        print(f"  Average frustration score: {df_results['frustration_score'].mean():.2f}")
        print(f"  Frustration score range: {df_results['frustration_score'].min():.2f} - {df_results['frustration_score'].max():.2f}")
        
        # Frustration level distribution
        level_counts = df_results['frustration_level'].value_counts()
        print(f"\\n😤 Frustration Levels:")
        for level, count in level_counts.items():
            percentage = count / len(frustration_results) * 100
            print(f"  {level.title()}: {count} ({percentage:.1f}%)")
        
        # Intervention recommendations
        intervention_needed = df_results['frustration_intervention_needed'].sum()
        print(f"\\n🚨 Intervention Recommendations:")
        print(f"  Human intervention needed: {intervention_needed} ({intervention_needed/len(frustration_results)*100:.1f}%)")
        
        # Escalation trends
        escalation_trends = df_results['escalation_trend'].value_counts()
        print(f"\\n📈 Escalation Trends:")
        for trend, count in escalation_trends.items():
            percentage = count / len(frustration_results) * 100
            print(f"  {trend.title()}: {count} ({percentage:.1f}%)")
        
        # Show sample analyses
        print(f"\\n📝 Sample Frustration Analyses:")
        for i, result in enumerate(frustration_results[:3]):
            print(f"\\n  Turn {result['conversation_id']}.{result['turn_number']} [{result['customer_type']}]:")
            print(f"    Frustration Score: {result['frustration_score']:.2f}/10.0")
            print(f"    Frustration Level: {result['frustration_level']}")
            print(f"    Confidence: {result['frustration_confidence']:.2f}")
            print(f"    Intervention Needed: {result['frustration_intervention_needed']}")
            factors = result['contributing_factors'][:3]
            if factors:
                print(f"    Contributing Factors: {', '.join(factors)}")
            print(f"    Query Score: {result['current_query_score']:.2f}, Pattern Score: {result['pattern_score']:.2f}")
            print(f"    Escalation Trend: {result['escalation_trend']}")
        
        if len(frustration_results) > 3:
            print(f"  ... and {len(frustration_results) - 3} more analyses")
            
        print(f"\\n✅ Frustration analysis data ready for detailed review and export!")
    
elif not conversation_data:
    print("⚠️ No conversation data loaded. Please upload conversation results in Step 3.")
elif not frustration_agent:
    print("⚠️ Frustration Agent not initialized. Please run Step 4 first.")
else:
    print("⚠️ Unable to start frustration analysis. Please check previous steps.")

🚀 Starting frustration analysis process...
😤 Running frustration analysis on 66 conversation turns...
\rProcessing turn 1/66 (Conv 1.1)...✅ 20:47:21.614 [INFO    ] claude-3-5-sonnet-20241022 | Model call: claude-3-5-sonnet-20241022 - generate_response | model_name=claude-3-5-sonnet-20241022 operation=generate_response
❌ 20:47:21.615 [ERROR   ] context_manager | Failed to get recent context | user_id=user_1 session_id=session_1 operation=get_recent_context
❌ 20:47:21.616 [ERROR   ] context_manager | Failed to save context entry | user_id=user_1 session_id=session_1 operation=save_context_entry
✅ 20:47:21.617 [INFO    ] frustration_agent | Frustration analysis completed | operation=frustration_analysis query_id=frustration_test_1_1 user_id=user_1 session_id=session_1
❌ 20:47:21.618 [ERROR   ] context_manager | Failed to save context entry | user_id=user_1 session_id=session_1 operation=save_context_entry
❌ 20:47:21.619 [ERROR   ] context_manager | Failed to save context entry | user_id=u

## Step 6: Analyze Frustration Patterns and Results

Detailed analysis of frustration patterns, escalation trends, and employee wellbeing recommendations.

In [8]:
# Detailed analysis of frustration detection results
if frustration_results:
    print("😤 Frustration Analysis Dashboard")
    print("=" * 60)
    
    # Create DataFrame for analysis
    df_frustration = pd.DataFrame(frustration_results)
    
    # Overall Frustration Statistics
    print(f"\n🎯 OVERALL FRUSTRATION STATISTICS")
    print(f"Total turns analyzed: {len(df_frustration)}")
    print(f"Mean frustration score: {df_frustration['frustration_score'].mean():.2f}/10.0")
    print(f"Median frustration score: {df_frustration['frustration_score'].median():.2f}/10.0")
    print(f"Standard deviation: {df_frustration['frustration_score'].std():.2f}")
    print(f"Score range: {df_frustration['frustration_score'].min():.2f} - {df_frustration['frustration_score'].max():.2f}")
    
    # Frustration Level Breakdown
    print(f"\n😤 FRUSTRATION LEVEL BREAKDOWN")
    level_analysis = df_frustration['frustration_level'].value_counts()
    for level, count in level_analysis.items():
        percentage = count / len(df_frustration) * 100
        print(f"  {level.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
    
    # Intervention Analysis
    print(f"\n🚨 INTERVENTION ANALYSIS")
    intervention_needed = df_frustration['frustration_intervention_needed'].sum()
    intervention_rate = intervention_needed / len(df_frustration) * 100
    print(f"  Human intervention recommended: {intervention_needed} ({intervention_rate:.1f}%)")
    print(f"  Customers continuing with AI: {len(df_frustration) - intervention_needed} ({100-intervention_rate:.1f}%)")
    
    # Frustration by Customer Type
    print(f"\n👥 FRUSTRATION BY CUSTOMER TYPE")
    for customer_type in df_frustration['customer_type'].unique():
        type_data = df_frustration[df_frustration['customer_type'] == customer_type]
        avg_score = type_data['frustration_score'].mean()
        avg_confidence = type_data['frustration_confidence'].mean()
        intervention_rate = type_data['frustration_intervention_needed'].sum() / len(type_data) * 100
        high_frustration_rate = len(type_data[type_data['frustration_level'].isin(['high', 'critical'])]) / len(type_data) * 100
        print(f"  {customer_type.title()}:")
        print(f"    Average frustration: {avg_score:.2f}/10.0")
        print(f"    Average confidence: {avg_confidence:.2f}")
        print(f"    Intervention rate: {intervention_rate:.1f}%")
        print(f"    High/Critical frustration rate: {high_frustration_rate:.1f}%")
        print(f"    Sample size: {len(type_data)}")
    
    # Frustration by Complexity
    print(f"\n🔢 FRUSTRATION BY COMPLEXITY LEVEL")
    for complexity in df_frustration['complexity'].unique():
        complexity_data = df_frustration[df_frustration['complexity'] == complexity]
        avg_score = complexity_data['frustration_score'].mean()
        avg_confidence = complexity_data['frustration_confidence'].mean()
        intervention_rate = complexity_data['frustration_intervention_needed'].sum() / len(complexity_data) * 100
        print(f"  {complexity.title()} Complexity:")
        print(f"    Average frustration: {avg_score:.2f}/10.0")
        print(f"    Average confidence: {avg_confidence:.2f}")
        print(f"    Intervention rate: {intervention_rate:.1f}%")
        print(f"    Sample size: {len(complexity_data)}")
    
    # Escalation Trend Analysis
    print(f"\n📈 ESCALATION TREND ANALYSIS")
    escalation_analysis = df_frustration['escalation_trend'].value_counts()
    for trend, count in escalation_analysis.items():
        percentage = count / len(df_frustration) * 100
        print(f"  {trend.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
    
    # Show frustration distribution with enhanced ranges
    print(f"\n📊 FRUSTRATION SCORE DISTRIBUTION")
    frustration_ranges = [
        (8.0, 10.0, "Critical", "🚨"),
        (6.0, 7.99, "High", "😠"),
        (4.0, 5.99, "Moderate", "😤"),
        (2.0, 3.99, "Low", "😐"),
        (0.0, 1.99, "Minimal", "😊")
    ]
    
    for min_score, max_score, label, emoji in frustration_ranges:
        count = len(df_frustration[(df_frustration['frustration_score'] >= min_score) & (df_frustration['frustration_score'] <= max_score)])
        percentage = count / len(df_frustration) * 100
        print(f"  {emoji} {label} ({min_score}-{max_score}): {count} ({percentage:.1f}%)")
    
    # Contributing Factors Analysis
    print(f"\n🔍 CONTRIBUTING FACTORS ANALYSIS")
    all_factors = []
    for result in frustration_results:
        all_factors.extend(result.get('contributing_factors', []))
    
    if all_factors:
        factor_counts = pd.Series(all_factors).value_counts()
        print(f"  Top contributing factors:")
        for factor, count in factor_counts.head(8).items():
            percentage = count / len(frustration_results) * 100
            print(f"    • {factor.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
    else:
        print(f"  No contributing factors data available")
    
    # Pattern vs Current Query Analysis
    print(f"\n🔄 PATTERN vs CURRENT QUERY ANALYSIS")
    high_pattern = len(df_frustration[df_frustration['pattern_score'] >= 6.0])
    high_current = len(df_frustration[df_frustration['current_query_score'] >= 6.0])
    both_high = len(df_frustration[(df_frustration['pattern_score'] >= 6.0) & (df_frustration['current_query_score'] >= 6.0)])
    
    print(f"  High pattern frustration (≥6.0): {high_pattern} ({high_pattern/len(df_frustration)*100:.1f}%)")
    print(f"  High current query frustration (≥6.0): {high_current} ({high_current/len(df_frustration)*100:.1f}%)")
    print(f"  Both pattern and current high: {both_high} ({both_high/len(df_frustration)*100:.1f}%)")
    
    # Confidence vs Score Analysis
    print(f"\n🎯 CONFIDENCE vs FRUSTRATION CORRELATION")
    high_conf_high_frust = len(df_frustration[(df_frustration['frustration_confidence'] >= 0.8) & (df_frustration['frustration_score'] >= 6.0)])
    low_conf_low_frust = len(df_frustration[(df_frustration['frustration_confidence'] <= 0.5) & (df_frustration['frustration_score'] <= 3.0)])
    print(f"High confidence + High frustration: {high_conf_high_frust} ({high_conf_high_frust/len(df_frustration)*100:.1f}%)")
    print(f"Low confidence + Low frustration: {low_conf_low_frust} ({low_conf_low_frust/len(df_frustration)*100:.1f}%)")
    
    # Show detailed examples by frustration level
    print(f"\n📝 DETAILED EXAMPLES BY FRUSTRATION LEVEL")
    
    # Critical frustration examples
    critical_examples = df_frustration[df_frustration['frustration_level'] == 'critical'].head(2)
    if len(critical_examples) > 0:
        print(f"\n🚨 CRITICAL FRUSTRATION:")
        for idx, row in critical_examples.iterrows():
            print(f"  Example {row['conversation_id']}.{row['turn_number']} (Score: {row['frustration_score']:.1f}):")
            print(f"    Query: {row['customer_query'][:100]}...")
            if row['contributing_factors']:
                factors_str = ', '.join(row['contributing_factors'][:3])
                print(f"    Factors: {factors_str}")
            print(f"    Trend: {row['escalation_trend']}, Intervention: {row['frustration_intervention_needed']}")
            print(f"    Confidence: {row['frustration_confidence']:.2f}")
    
    # High frustration examples
    high_examples = df_frustration[df_frustration['frustration_level'] == 'high'].head(2)
    if len(high_examples) > 0:
        print(f"\n😠 HIGH FRUSTRATION:")
        for idx, row in high_examples.iterrows():
            print(f"  Example {row['conversation_id']}.{row['turn_number']} (Score: {row['frustration_score']:.1f}):")
            print(f"    Query: {row['customer_query'][:100]}...")
            if row['contributing_factors']:
                factors_str = ', '.join(row['contributing_factors'][:3])
                print(f"    Factors: {factors_str}")
            print(f"    Trend: {row['escalation_trend']}, Intervention: {row['frustration_intervention_needed']}")
            print(f"    Confidence: {row['frustration_confidence']:.2f}")
    
    # Moderate frustration examples
    moderate_examples = df_frustration[df_frustration['frustration_level'] == 'moderate'].head(2)
    if len(moderate_examples) > 0:
        print(f"\n😤 MODERATE FRUSTRATION:")
        for idx, row in moderate_examples.iterrows():
            print(f"  Example {row['conversation_id']}.{row['turn_number']} (Score: {row['frustration_score']:.1f}):")
            print(f"    Query: {row['customer_query'][:100]}...")
            if row['contributing_factors']:
                factors_str = ', '.join(row['contributing_factors'][:3])
                print(f"    Factors: {factors_str}")
            print(f"    Trend: {row['escalation_trend']}, Intervention: {row['frustration_intervention_needed']}")
            print(f"    Confidence: {row['frustration_confidence']:.2f}")
    
    # Low frustration examples for comparison
    low_examples = df_frustration[df_frustration['frustration_level'].isin(['low', 'minimal'])].head(2)
    if len(low_examples) > 0:
        print(f"\n😊 LOW/MINIMAL FRUSTRATION:")
        for idx, row in low_examples.iterrows():
            print(f"  Example {row['conversation_id']}.{row['turn_number']} (Score: {row['frustration_score']:.1f}):")
            print(f"    Query: {row['customer_query'][:100]}...")
            print(f"    Level: {row['frustration_level']}, Confidence: {row['frustration_confidence']:.2f}")
    
    # Employee Wellbeing Impact Analysis
    print(f"\n👥 EMPLOYEE WELLBEING IMPACT ANALYSIS")
    critical_high_count = len(df_frustration[df_frustration['frustration_level'].isin(['critical', 'high'])])
    escalating_count = len(df_frustration[df_frustration['escalation_trend'] == 'escalating'])
    
    print(f"  High-stress customer interactions: {critical_high_count} ({critical_high_count/len(df_frustration)*100:.1f}%)")
    print(f"  Escalating frustration patterns: {escalating_count} ({escalating_count/len(df_frustration)*100:.1f}%)")
    print(f"  Recommended for experienced agents: {intervention_needed}")
    
    if critical_high_count > 0:
        print(f"\n💡 Employee Protection Recommendations:")
        print(f"  • Rotate high-frustration customers among multiple agents")
        print(f"  • Provide emotional support for agents handling {critical_high_count} difficult cases")
        print(f"  • Consider priority routing for {escalating_count} escalating situations")
        print(f"  • Monitor agent stress levels during peak frustration periods")
        print(f"  • Implement cooldown periods after handling critical frustration cases")
        print(f"  • Provide specialized training for {intervention_needed} intervention-required cases")
    
    # Frustration Progression Analysis
    print(f"\n📊 FRUSTRATION PROGRESSION ANALYSIS")
    multi_turn_convs = df_frustration[df_frustration.groupby('conversation_id')['conversation_id'].transform('count') > 1]
    if len(multi_turn_convs) > 0:
        print(f"  Multi-turn conversations: {multi_turn_convs['conversation_id'].nunique()}")
        
        # Analyze progression patterns
        escalating_convs = multi_turn_convs[multi_turn_convs['escalation_trend'] == 'escalating']['conversation_id'].nunique()
        stable_convs = multi_turn_convs[multi_turn_convs['escalation_trend'] == 'stable']['conversation_id'].nunique()
        improving_convs = multi_turn_convs[multi_turn_convs['escalation_trend'] == 'improving']['conversation_id'].nunique()
        
        total_multi = multi_turn_convs['conversation_id'].nunique()
        print(f"  Escalating patterns: {escalating_convs} ({escalating_convs/total_multi*100:.1f}%)")
        print(f"  Stable patterns: {stable_convs} ({stable_convs/total_multi*100:.1f}%)")
        print(f"  Improving patterns: {improving_convs} ({improving_convs/total_multi*100:.1f}%)")
    else:
        print(f"  All conversations are single-turn")
    
    # Show summary table with enhanced columns
    print(f"\n📊 FRUSTRATION ANALYSIS SUMMARY TABLE")
    summary_df = df_frustration[['conversation_id', 'turn_number', 'customer_type', 'complexity', 
                               'frustration_score', 'frustration_level', 'frustration_confidence', 
                               'frustration_intervention_needed', 'escalation_trend', 
                               'current_query_score', 'pattern_score']].copy()
    summary_df['frustration_score'] = summary_df['frustration_score'].round(1)
    summary_df['frustration_confidence'] = summary_df['frustration_confidence'].round(2)
    summary_df['current_query_score'] = summary_df['current_query_score'].round(1)
    summary_df['pattern_score'] = summary_df['pattern_score'].round(1)
    
    display(summary_df.head(10))
    
    if len(summary_df) > 10:
        print(f"... showing first 10 of {len(summary_df)} analyzed turns")
    
    # Advanced Analytics Summary
    print(f"\n📈 ADVANCED ANALYTICS SUMMARY")
    avg_conf = df_frustration['frustration_confidence'].mean()
    high_confidence_rate = len(df_frustration[df_frustration['frustration_confidence'] >= 0.8]) / len(df_frustration) * 100
    consistent_analysis = len(df_frustration[abs(df_frustration['current_query_score'] - df_frustration['pattern_score']) <= 2.0]) / len(df_frustration) * 100
    
    print(f"  Average confidence: {avg_conf:.2f}")
    print(f"  High confidence detections (≥0.8): {high_confidence_rate:.1f}%")
    print(f"  Consistent pattern-query analysis (≤2.0 difference): {consistent_analysis:.1f}%")
    
    print(f"\n" + "="*60)
    print(f"✅ Comprehensive frustration analysis complete! Use Step 7 to export detailed results.")

else:
    print("⚠️ No frustration analysis results to review. Please run frustration analysis in Step 5 first.")

😤 Frustration Analysis Dashboard

🎯 OVERALL FRUSTRATION STATISTICS
Total turns analyzed: 66
Mean frustration score: 1.81/10.0
Median frustration score: 1.75/10.0
Standard deviation: 0.98
Score range: 0.35 - 4.38

😤 FRUSTRATION LEVEL BREAKDOWN
  Low: 60 (90.9%)
  Moderate: 6 (9.1%)

🚨 INTERVENTION ANALYSIS
  Human intervention recommended: 0 (0.0%)
  Customers continuing with AI: 66 (100.0%)

👥 FRUSTRATION BY CUSTOMER TYPE
  Frustrated:
    Average frustration: 2.80/10.0
    Average confidence: 0.57
    Intervention rate: 0.0%
    High/Critical frustration rate: 0.0%
    Sample size: 11
  Urgent:
    Average frustration: 1.69/10.0
    Average confidence: 0.54
    Intervention rate: 0.0%
    High/Critical frustration rate: 0.0%
    Sample size: 18
  Confused:
    Average frustration: 1.84/10.0
    Average confidence: 0.54
    Intervention rate: 0.0%
    High/Critical frustration rate: 0.0%
    Sample size: 25
  Normal:
    Average frustration: 1.01/10.0
    Average confidence: 0.52
    I

,conversation_id,turn_number,customer_type,complexity,frustration_score,frustration_level,frustration_confidence,frustration_intervention_needed,escalation_trend,current_query_score,pattern_score
0,1,1,frustrated,medium,2.8,low,0.6,False,none,4.0,0.0
1,1,2,frustrated,medium,4.4,moderate,0.6,False,none,6.2,0.0
2,1,3,frustrated,medium,1.8,low,0.5,False,none,2.5,0.0
3,2,1,urgent,simple,1.6,low,0.6,False,none,2.2,0.0
4,2,2,urgent,simple,1.8,low,0.5,False,none,2.5,0.0
5,2,3,urgent,simple,2.3,low,0.6,False,none,3.2,0.0
6,3,1,confused,simple,2.6,low,0.6,False,none,3.8,0.0
7,3,2,confused,simple,1.8,low,0.5,False,none,2.5,0.0
8,3,3,confused,simple,2.4,low,0.6,False,none,3.5,0.0
9,3,4,confused,simple,1.9,low,0.6,False,none,2.8,0.0


... showing first 10 of 66 analyzed turns

📈 ADVANCED ANALYTICS SUMMARY
  Average confidence: 0.54
  High confidence detections (≥0.8): 0.0%
  Consistent pattern-query analysis (≤2.0 difference): 22.7%

✅ Comprehensive frustration analysis complete! Use Step 7 to export detailed results.


## Step 7: Export Frustration Analysis Results

Save the enhanced conversation data with frustration metrics and employee wellbeing recommendations to files with timestamps.

In [10]:
# Export frustration analysis results and settings to files with timestamps
def export_frustration_results():
    """Export frustration analysis results and configuration to JSON files"""
    if not frustration_results:
        print("❌ No frustration analysis results to export.")
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = Path(f'/workspace/notebooks/agent_testers/outputs/frustration_evals/{timestamp}')
    output_dir.mkdir(exist_ok=True)
    
    
    # Prepare export data
    export_data = {
        'metadata': {
            'export_timestamp': datetime.now().isoformat(),
            'frustration_agent_version': frustration_agent.agent_config.version if frustration_agent.agent_config else '1.0.0',
            'total_turns_analyzed': len(frustration_results),
            'average_frustration_score': sum(r['frustration_score'] for r in frustration_results) / len(frustration_results),
            'analysis_model': frustration_agent.llm_provider.model_name if frustration_agent.llm_provider else 'unknown',
            'frustration_thresholds': frustration_agent.agent_config.settings.get('frustration_thresholds', {}) if frustration_agent.agent_config else {},
            'intervention_threshold': frustration_agent.agent_config.settings.get('intervention_threshold', 'high') if frustration_agent.agent_config else 'high'
        },
        'frustration_analyses': frustration_results,
        'summary_statistics': {
            'level_distribution': dict(pd.DataFrame(frustration_results)['frustration_level'].value_counts()),
            'escalation_trends': dict(pd.DataFrame(frustration_results)['escalation_trend'].value_counts()),
            'intervention_statistics': {
                'total_interventions_recommended': sum(1 for r in frustration_results if r['frustration_intervention_needed']),
                'intervention_rate': sum(1 for r in frustration_results if r['frustration_intervention_needed']) / len(frustration_results),
            },
            'score_statistics': {
                'mean': sum(r['frustration_score'] for r in frustration_results) / len(frustration_results),
                'median': sorted([r['frustration_score'] for r in frustration_results])[len(frustration_results)//2],
                'min': min(r['frustration_score'] for r in frustration_results),
                'max': max(r['frustration_score'] for r in frustration_results)
            },
            'customer_type_analysis': {},
            'complexity_analysis': {},
            'employee_wellbeing_impact': {}
        }
    }
    
    # Add customer type analysis
    df_frustration = pd.DataFrame(frustration_results)
    for customer_type in df_frustration['customer_type'].unique():
        type_data = df_frustration[df_frustration['customer_type'] == customer_type]
        export_data['summary_statistics']['customer_type_analysis'][customer_type] = {
            'count': len(type_data),
            'average_frustration_score': type_data['frustration_score'].mean(),
            'intervention_rate': type_data['frustration_intervention_needed'].sum() / len(type_data),
            'high_frustration_rate': len(type_data[type_data['frustration_level'].isin(['high', 'critical'])]) / len(type_data)
        }
    
    # Add complexity analysis
    for complexity in df_frustration['complexity'].unique():
        complexity_data = df_frustration[df_frustration['complexity'] == complexity]
        export_data['summary_statistics']['complexity_analysis'][complexity] = {
            'count': len(complexity_data),
            'average_frustration_score': complexity_data['frustration_score'].mean(),
            'intervention_rate': complexity_data['frustration_intervention_needed'].sum() / len(complexity_data)
        }
    
    # Add employee wellbeing analysis
    critical_high_count = len(df_frustration[df_frustration['frustration_level'].isin(['critical', 'high'])])
    escalating_count = len(df_frustration[df_frustration['escalation_trend'] == 'escalating'])
    
    export_data['summary_statistics']['employee_wellbeing_impact'] = {
        'high_stress_interactions': critical_high_count,
        'high_stress_rate': critical_high_count / len(df_frustration),
        'escalating_patterns': escalating_count,
        'escalating_rate': escalating_count / len(df_frustration),
        'recommendations': [
            'Rotate high-frustration customers among multiple agents',
            'Provide emotional support for agents handling difficult cases',
            'Monitor agent stress levels during peak frustration periods',
            'Consider priority routing for escalating situations'
        ]
    }
    
    # Export main results
    results_filename = f'frustration_analysis_results.json'
    results_path = output_dir / results_filename
    
    with open(results_path, 'w') as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"✅ Frustration analysis results exported to: {results_path}")
    
    # Export configuration used
    config_export = {
        'export_timestamp': datetime.now().isoformat(),
        'agent_config': {},
        'prompts_config': {},
        'models_config': {}
    }
    
    # Read current config contents
    try:
        with open(temp_file_paths['agent_config'], 'r') as f:
            yaml = YAML()
            config_export['agent_config'] = yaml.load(f)
    except:
        pass
    
    try:
        with open(temp_file_paths['prompts_config'], 'r') as f:
            yaml = YAML()
            config_export['prompts_config'] = yaml.load(f)
    except:
        pass
    
    try:
        with open(temp_file_paths['models_config'], 'r') as f:
            yaml = YAML()
            config_export['models_config'] = yaml.load(f)
    except:
        pass
    
    config_filename = f'frustration_agent_config.json'
    config_path = output_dir / config_filename
    
    with open(config_path, 'w') as f:
        json.dump(config_export, f, indent=2, default=str)
    
    print(f"✅ Configuration settings exported to: {config_path}")
    
    # Export summary CSV for easy analysis
    summary_df = pd.DataFrame(frustration_results)
    csv_filename = f'frustration_summary.csv'
    csv_path = output_dir / csv_filename
    
    # Select key columns for CSV
    csv_columns = ['conversation_id', 'turn_number', 'customer_type', 'complexity',
                   'frustration_score', 'frustration_level', 'frustration_confidence', 
                   'frustration_intervention_needed', 'escalation_trend', 'contributing_factors',
                   'current_query_score', 'pattern_score', 'customer_query']
    
    summary_csv = summary_df[csv_columns].copy()
    # Convert list columns to strings for CSV
    summary_csv['contributing_factors'] = summary_csv['contributing_factors'].apply(lambda x: '; '.join(x) if x else '')
    summary_csv.to_csv(csv_path, index=False)
    
    print(f"✅ Summary CSV exported to: {csv_path}")
    
    # Generate frustration analysis report
    report_filename = f'frustration_report.txt'
    report_path = output_dir / report_filename
    
    with open(report_path, 'w') as f:
        f.write("FRUSTRATION ANALYSIS REPORT\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total Turns Analyzed: {len(frustration_results)}\n")
        f.write(f"Average Frustration Score: {export_data['metadata']['average_frustration_score']:.2f}/10.0\n")
        f.write(f"Analysis Model: {export_data['metadata']['analysis_model']}\n")
        f.write(f"Intervention Threshold: {export_data['metadata']['intervention_threshold']}\n\n")
        
        f.write("FRUSTRATION LEVEL DISTRIBUTION:\n")
        for level, count in export_data['summary_statistics']['level_distribution'].items():
            percentage = count / len(frustration_results) * 100
            f.write(f"  {level.replace('_', ' ').title()}: {count} ({percentage:.1f}%)\n")
        f.write("\n")
        
        f.write("INTERVENTION ANALYSIS:\n")
        intervention_stats = export_data['summary_statistics']['intervention_statistics']
        f.write(f"  Interventions Recommended: {intervention_stats['total_interventions_recommended']}\n")
        f.write(f"  Intervention Rate: {intervention_stats['intervention_rate']*100:.1f}%\n\n")
        
        f.write("ESCALATION TRENDS:\n")
        for trend, count in export_data['summary_statistics']['escalation_trends'].items():
            percentage = count / len(frustration_results) * 100
            f.write(f"  {trend.replace('_', ' ').title()}: {count} ({percentage:.1f}%)\n")
        f.write("\n")
        
        f.write("CUSTOMER TYPE ANALYSIS:\n")
        for customer_type, stats in export_data['summary_statistics']['customer_type_analysis'].items():
            f.write(f"  {customer_type.title()}:\n")
            f.write(f"    Sample Size: {stats['count']}\n")
            f.write(f"    Average Frustration: {stats['average_frustration_score']:.2f}/10.0\n")
            f.write(f"    Intervention Rate: {stats['intervention_rate']*100:.1f}%\n")
            f.write(f"    High Frustration Rate: {stats['high_frustration_rate']*100:.1f}%\n")
        f.write("\n")
        
        f.write("EMPLOYEE WELLBEING IMPACT:\n")
        wellbeing = export_data['summary_statistics']['employee_wellbeing_impact']
        f.write(f"  High-Stress Interactions: {wellbeing['high_stress_interactions']} ({wellbeing['high_stress_rate']*100:.1f}%)\n")
        f.write(f"  Escalating Patterns: {wellbeing['escalating_patterns']} ({wellbeing['escalating_rate']*100:.1f}%)\n")
        f.write("\n  Recommendations:\n")
        for rec in wellbeing['recommendations']:
            f.write(f"    • {rec}\n")
    
    print(f"✅ Frustration analysis report exported to: {report_path}")
    
    return {
        'results_file': str(results_path),
        'config_file': str(config_path),
        'csv_file': str(csv_path),
        'report_file': str(report_path)
    }

# Export results if available
if frustration_results:
    print("💾 Exporting Frustration Analysis Results and Configuration")
    print("=" * 60)
    
    export_files = export_frustration_results()
    
    if export_files:
        print(f"\n📁 All files exported successfully!")
        print(f"\n📋 Export Summary:")
        print(f"  Results JSON: {export_files['results_file']}")
        print(f"  Configuration: {export_files['config_file']}")
        print(f"  Summary CSV: {export_files['csv_file']}")
        print(f"  Frustration Report: {export_files['report_file']}")
        
        print(f"\n💡 File Usage:")
        print(f"  • Results JSON: Complete data for further analysis or integration")
        print(f"  • Configuration: Settings used for frustration detection")
        print(f"  • Summary CSV: Import into Excel, Google Sheets, or data analysis tools")
        print(f"  • Frustration Report: Human-readable summary for management review")
        
        print(f"\n🔄 Next Steps:")
        print(f"  • Use results to fine-tune frustration detection thresholds")
        print(f"  • Analyze patterns to improve customer experience")
        print(f"  • Implement employee wellbeing protection measures")
        print(f"  • Compare frustration trends across different configurations")
        print(f"  • Share analysis with customer service management")
        
    else:
        print("❌ Export failed. Please check for errors above.")

else:
    print("⚠️ No frustration analysis results to export. Please run frustration analysis in Step 5 first.")

💾 Exporting Frustration Analysis Results and Configuration
✅ Frustration analysis results exported to: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_analysis_results.json
✅ Configuration settings exported to: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_agent_config.json
✅ Summary CSV exported to: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_summary.csv
✅ Frustration analysis report exported to: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_report.txt

📁 All files exported successfully!

📋 Export Summary:
  Results JSON: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_analysis_results.json
  Configuration: /workspace/notebooks/agent_testers/outputs/frustration_evals/20250723_210436/frustration_agent_config.json
  Summary CSV: /workspace/notebooks/agent_testers/outputs/frustr